In [ ]:
import cv2
import numpy as np

from keras.models import load_model
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from keras.models import model_from_json

In [ ]:
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_data_gen.flow_from_directory(
	'data/train/',
	target_size=(48, 48),
	batch_size=64,
	color_mode="grayscale",
	class_mode='categorical'
)

Found 21005 images belonging to 4 classes.


In [ ]:
validation_generator = validation_data_gen.flow_from_directory(
		'data/test/',
		target_size=(48, 48),
		batch_size=64,
		color_mode="grayscale",
		class_mode='categorical'
)

Found 5212 images belonging to 4 classes.


In [ ]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))

emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(4, activation='softmax'))

emotion_model.summary()

cv2.ocl.setUseOpenCL(False)

initial_learning_rate = 0.0005
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=100000, decay_rate=0.96)

optimizer = Adam(learning_rate=lr_schedule)

emotion_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2  (None, 22, 22, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 128)       0         
 g2D)                                                   

In [ ]:
emotion_model_info = emotion_model.fit(
		train_generator,
		steps_per_epoch=21005 // 64,
		epochs=40,
		validation_data=validation_generator,
		validation_steps=5212 // 64
)

Epoch 1/40
328/328 [==============================] - 10s 30ms/step - loss: 1.0703 - accuracy: 0.5294 - val_loss: 0.9881 - val_accuracy: 0.5708
Epoch 2/40
328/328 [==============================] - 10s 30ms/step - loss: 0.9732 - accuracy: 0.5855 - val_loss: 0.9265 - val_accuracy: 0.6005
Epoch 3/40
328/328 [==============================] - 11s 32ms/step - loss: 0.9100 - accuracy: 0.6177 - val_loss: 0.8778 - val_accuracy: 0.6345
Epoch 4/40
328/328 [==============================] - 12s 36ms/step - loss: 0.8647 - accuracy: 0.6399 - val_loss: 0.8388 - val_accuracy: 0.6503
Epoch 5/40
328/328 [==============================] - 10s 31ms/step - loss: 0.8223 - accuracy: 0.6573 - val_loss: 0.8383 - val_accuracy: 0.6487
Epoch 6/40
328/328 [==============================] - 12s 36ms/step - loss: 0.7909 - accuracy: 0.6755 - val_loss: 0.8160 - val_accuracy: 0.6593
Epoch 7/40
328/328 [==============================] - 11s 33ms/step - loss: 0.7569 - accuracy: 0.6911 - val_loss: 0.8122 - val_accuracy:

In [ ]:
emotion_model.evaluate(validation_generator)

82/82 [==============================] - 2s 29ms/step - loss: 1.0947 - accuracy: 0.7001


[1.094722867012024, 0.7001151442527771]

In [ ]:
emotion_model.save('model_file.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
